In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from datetime import datetime
import time
import joblib
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.stats import loguniform
from scipy.stats import truncnorm
from scipy.stats import bernoulli

In [2]:
# Display all dataframe columns
pd.set_option('display.max_columns', None)

In [3]:
from sklearn.metrics import mean_squared_error

def rmse_precent(y_true, y_pred):
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # RMSE% rispetto alla media
    return (rmse / np.mean(y_true)) * 100

In [4]:
df = pd.read_csv("meleto_per_analisi.csv")
df.head(3)

,Treatment,target,botton_score,max,min,range,mean,std,median,non_zero_per_row
0,UTC,0.75,0.0430,16.3,13.1,3.2,15.0750,1.3769,15.45,4
1,UTC,0.33,0.1130,17.3,12.2,5.1,14.7333,2.5502,14.70,3
2,UTC,0.25,0.0406,15.6,12.7,2.9,14.2000,1.3342,14.25,4


In [5]:
print(df.apply(lambda col: col.unique()))

Treatment                                      [UTC, Brevis, BA, ACC]
target              [0.75, 0.33, 0.25, 0.5, 0.67, 1.0, 0.0, 0.4, 0...
botton_score        [0.043, 0.113, 0.0406, 0.0986, 0.0447, 0.1202,...
max                 [16.3, 17.3, 15.6, 16.4, 15.7, 15.8, 15.1, 13....
min                 [13.1, 12.2, 12.7, 9.0, 11.9, 11.0, 9.5, 7.5, ...
range               [3.2, 5.1, 2.9, 7.4, 3.8, 4.8, 7.8, 7.6, 4.9, ...
mean                [15.075, 14.7333, 14.2, 13.5, 13.3667, 13.6333...
std                 [1.3769, 2.5502, 1.3342, 3.3882, 2.0429, 2.433...
median              [15.45, 14.7, 14.25, 14.3, 12.5, 14.1, 16.7, 9...
non_zero_per_row                                [4, 3, 1, 5, 2, 6, 7]
dtype: object


In [6]:
# get treatment type for train_test_split stratify    
treatment = df['Treatment'] 

df = df.drop(columns=['Treatment'])
display(df.head(3))

,target,botton_score,max,min,range,mean,std,median,non_zero_per_row
0,0.75,0.0430,16.3,13.1,3.2,15.0750,1.3769,15.45,4
1,0.33,0.1130,17.3,12.2,5.1,14.7333,2.5502,14.70,3
2,0.25,0.0406,15.6,12.7,2.9,14.2000,1.3342,14.25,4


In [7]:
y = df['target'] 
X = df.drop(['target'], axis=1) # drop target
display(X.head(3))

,botton_score,max,min,range,mean,std,median,non_zero_per_row
0,0.0430,16.3,13.1,3.2,15.0750,1.3769,15.45,4
1,0.1130,17.3,12.2,5.1,14.7333,2.5502,14.70,3
2,0.0406,15.6,12.7,2.9,14.2000,1.3342,14.25,4


In [8]:
# 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=treatment)

In [9]:
# Remove botton score
X_train = X_train.drop(['botton_score'], axis=1)

# remove botton score
botton_score_test = X_test['botton_score']
X_test = X_test.drop(['botton_score'], axis=1)

# display for check
display(X_train.head(3))
display(X_test.head(3))

,max,min,range,mean,std,median,non_zero_per_row
4996,22.07,8.93,13.14,17.7160,5.2712,18.76,5
4965,19.26,9.76,9.50,15.3967,4.1283,17.15,6
955,23.67,8.62,15.05,16.7200,7.5906,17.87,3


,max,min,range,mean,std,median,non_zero_per_row
4914,14.00,12.00,2.00,13.200,0.8367,13.00,5
7008,15.00,11.70,3.30,13.260,1.3502,13.30,5
1373,12.33,6.08,6.25,9.294,2.2648,9.42,5


In [10]:
# Metrics for botton score for all sample no treatment
metrics_botton_score = {
    'Model': 'Botton score',
    'MAE': mean_absolute_error(y_test, botton_score_test), # Mean Absolute Error
    'MSE': mean_squared_error(y_test, botton_score_test), # Mean Squared Error
    'RMSE': root_mean_squared_error(y_test, botton_score_test), # Root Mean Squared Error
    'RMSE_per': rmse_precent(y_test, botton_score_test), # Root Mean Squared Error %    
    'R2': r2_score(y_test, botton_score_test), # R2
    'Median_AE': median_absolute_error(y_test, botton_score_test) # Median Absolute Error
}

## XGB Regression for all sample and no treatment as feature DART

In [11]:
# media = 0.15, deviazione = 0.05, truncato tra 0 e 1
loc = 0.15
scale = 0.05
a, b = (0 - loc) / scale, (1 - loc) / scale
truncated = truncnorm(a, b, loc=loc, scale=scale)

In [12]:
# Define hyperparameter distribution for xgboost
param_dist = {
    'n_estimators': randint(500, 1500), # integer
    'max_depth': randint(3, 10), # integer
    'min_child_weight': randint(1, 50), # integer
    'subsample': uniform(loc=0.5, scale=0.5), # float loc=0.5 (inizio intervallo) e scale=0.5 (lunghezza intervallo), 0-5 + 0.5 = 1
    'colsample_bytree': uniform(loc=0.5, scale=0.5), # float
    'learning_rate': beta(a=2, b=1, loc=0.05, scale=0.15), # Valori tra 0.1 e 0.2 saranno più probabili di quelli vicino a 0.05.
    'gamma': loguniform(1e-2, 10),  # più densità vicino a 0.01–1, meno verso 10
    'alpha': loguniform(1e-3, 10),  # 0.001 – 10 la maggior parte delle volte i valori ottimali sono bassi (vicino a 0–1), ma conviene testare anche numeri più grandi.
    'lambda': loguniform(1e-3, 10)  # da 0.001 a 10
}

In [ ]:
booster = "dart"

model = XGBRegressor(
    booster=booster,
    tree_method='exact',
    n_jobs=-1)

# Perform randomized search
xgb_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=1000,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1
    # random_state=42
)

# Fit the model 
xgb_search.fit(X_train, y_train.squeeze())

# Miglior modello e predizioni
best_model = xgb_search.best_estimator_ # BEST MODEL
y_pred = best_model.predict(X_test)

# Metrics
metrics = {
    'Model': 'XGBoost',
    'MAE': mean_absolute_error(y_test, y_pred), # Mean Absolute Error
    'MSE': mean_squared_error(y_test, y_pred), # Mean Squared Error
    'RMSE': root_mean_squared_error(y_test, y_pred), # Root Mean Squared Error
    'RMSE_per': rmse_precent(y_test, y_pred), # Root Mean Squared Error %    
    'R2': r2_score(y_test, y_pred), # R^2
    'Median_AE': median_absolute_error(y_test, y_pred) # Median Absolute Error
}

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


In [ ]:
# Save all
timestamp = datetime.now().strftime("%Y%m%dT%H%M") # Timestamp per i file
    
# (1) Save Best Model
joblib.dump(best_model, f"XGB_{booster}_NoTreatment_best_model_{timestamp}.pkl")
    
# (2) Save Metrics
df_metrics = pd.DataFrame([metrics_botton_score, metrics])  # una riga con tutte le metriche
df_metrics.to_csv(f"XGB_{booster}_NoTreatment_evaluation_metrics_{timestamp}.csv", index=False)
    
# (3) Save best hyper-parameters
best_params = xgb_search.best_params_ # get best hyper-parameters
df_params = pd.DataFrame([best_params])  # una riga con tutti gli iperparametri
df_params.to_csv(f"XGB_{booster}_NoTreatment_best_params_{timestamp}.csv", index=False)
    
# (4) Save y_test e y_pred
df_pred = pd.DataFrame({'y_test': y_test.squeeze(), 'y_pred': y_pred})
df_pred.to_csv(f"XGB_{booster}_NoTreatment_predictions_{timestamp}.csv", index=False)

In [ ]:
print(f"Errors Botton score all sample No Treatment:")
print(f" - MAE: {round(metrics_botton_score['MAE'], 4)}")
print(f" - MSE: {round(metrics_botton_score['MSE'], 4)}")
print(f" - RMSE: {round(metrics_botton_score['RMSE'], 4)}")
print(f" - RMSE_per: {round(metrics_botton_score['RMSE_per'], 4)}")
print(f" - R2: {round(metrics_botton_score['R2'], 4)}")
print(f" - Median Absolute Error: {round(metrics_botton_score['Median_AE'], 4)}")

In [ ]:
print(f"Errors XGboost {booster} all sample No Treatment:")
print(f" - MAE: {round(metrics['MAE'], 4)}")
print(f" - MSE: {round(metrics['MSE'], 4)}")
print(f" - RMSE: {round(metrics['RMSE'], 4)}")
print(f" - RMSE_per: {round(metrics['RMSE_per'], 4)}")
print(f" - R2: {round(metrics['R2'], 4)}")
print(f" - Median Absolute Error: {round(metrics['Median_AE'], 4)}")

In [ ]:
importance = best_model.get_booster().get_score(importance_type='gain')

# converti in DataFrame
importance_df = pd.DataFrame({
    'Feature': list(importance.keys()),
    'Importance': list(importance.values())
})

# ordina in ordine decrescente
feat_importances = importance_df.sort_values(by='Importance', ascending=False)
print("\nFeature importance:\n", feat_importances)

feat_importances.to_csv(f"XGB_{booster}_NoTreatment_FeatureImportance_{timestamp}.csv", index=False)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(feat_importances["Feature"], feat_importances["Importance"], color="skyblue", edgecolor="black")
plt.title(f"Feature Importance - Best XGB Model {booster} - all samples but without treatment as feature")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.gca().invert_yaxis() 
plt.tight_layout()
plt.show()